In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [207]:
df = pd.read_csv('C:\\Users\\Suraj\Downloads\\Car details updated.csv')

In [206]:
df.head()

,Sales_ID,name,year,selling_price,km_driven,Region,State or Province,City,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,sold
0,1,Maruti Swift Dzire VDI,2014,450000,145500,East,District of Columbia,Washington,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5,Y
1,2,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,East,New York,New York City,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5,Y
2,3,Honda City 2017-2020 EXi,2006,158000,140000,Central,Illinois,Chicago,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5,Y
3,4,Hyundai i20 Sportz Diesel,2010,225000,127000,Central,Illinois,Chicago,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5,Y
4,5,Maruti Swift VXI BSIII,2007,130000,120000,East,New York,New York City,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5,Y


In [4]:
df.shape

(7906, 18)

In [5]:
df['sold'].value_counts()

N    5906
Y    2000
Name: sold, dtype: int64

In [8]:
df['City'].nunique()

1310

In [9]:
df['Region'].value_counts()

Central    2376
West       1976
East       1939
South      1615
Name: Region, dtype: int64

In [12]:
df['State or Province'].groupby(df['State or Province']).transform('count')

0        68
1       491
2       489
3       489
4       491
       ... 
7901    499
7902    499
7903    499
7904    191
7905    110
Name: State or Province, Length: 7906, dtype: int64

In [21]:
df['mileage'] = pd.to_numeric(df['mileage'].str.split(" ")[0][0])

In [66]:
df.head()

,year,selling_price,km_driven,Region,State or Province,City,fuel,seller_type,transmission,owner,mileage,engine,max_power,seats,sold,torque_NewtonMeters,company_name
0,2014,450000,145500,East,District of Columbia,Washington,Diesel,Individual,0,First Owner,23.4,1248,74,5,1,190,Maruti
1,2014,370000,120000,East,New York,New York City,Diesel,Individual,0,Second Owner,23.4,1248,74,5,1,250,Skoda
2,2006,158000,140000,Central,Illinois,Chicago,Petrol,Individual,0,Third Owner,23.4,1248,74,5,1,12,Honda
3,2010,225000,127000,Central,Illinois,Chicago,Diesel,Individual,0,First Owner,23.4,1248,74,5,1,22,Hyundai
4,2007,130000,120000,East,New York,New York City,Petrol,Individual,0,First Owner,23.4,1248,74,5,1,11,Maruti


### Data Preprocessing

In [23]:
df['engine'] = pd.to_numeric(df['engine'].str.split(" ")[0][0])

In [25]:
df['max_power'] = pd.to_numeric(df['max_power'].str.split(" ")[0][0])

In [36]:
import re
df['torque_NewtonMeters'] = df['torque'].apply(lambda x: pd.to_numeric(re.search('\d+', x).group()))

In [38]:
df.drop('torque',axis=1,inplace=True)

In [40]:
df.drop('Sales_ID',axis=1,inplace=True)

In [137]:
df['owner'].value_counts()

First Owner             5215
Second Owner            2016
Third Owner              510
Fourth & Above Owner     160
Test Drive Car             5
Name: owner, dtype: int64

In [51]:
print(df['name'].str.split(" ")[0][0])

Maruti


In [58]:
df['company_name'] = df['name'].apply(lambda x: x.split(" ")[0])

In [60]:
df.drop('name',axis=1,inplace=True)

In [63]:
df['Region'].value_counts()

Central    2376
West       1976
East       1939
South      1615
Name: Region, dtype: int64

In [65]:
df['transmission'] = df['transmission'].map({'Manual':0,'Automatic':1})
df['sold'] = df['sold'].map({'N':0,'Y':1})

In [69]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import FeatureHasher

In [71]:
X = df.drop('sold',axis=1)
y = df['sold']

In [111]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,stratify = y,random_state=42)

In [112]:
X_train.shape,y_train.shape

((6324, 16), (6324,))

In [113]:
X_test.shape,y_test.shape

((1582, 16), (1582,))

In [ ]:
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)

In [177]:
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [114]:
ohe = OneHotEncoder()
Xtrainfuel = ohe.fit_transform(X_train[['fuel']]).toarray()
Xtestfuel = ohe.transform(X_test[['fuel']]).toarray()

In [117]:
ohe.get_feature_names()

array(['x0_CNG', 'x0_Diesel', 'x0_LPG', 'x0_Petrol'], dtype=object)

In [118]:
Xtrainfuel = pd.DataFrame(Xtrainfuel,columns = ohe.get_feature_names())
Xtestfuel = pd.DataFrame(Xtestfuel,columns = ohe.get_feature_names())

In [124]:
X_train = pd.concat([X_train,Xtrainfuel],axis=1)
X_test = pd.concat([X_test,Xtestfuel],axis=1)

In [131]:
X_train.drop('fuel',axis=1,inplace=True)
X_test.drop('fuel',axis=1,inplace=True)

In [148]:
def oheencode(feat_name,X_train,X_test):
    ohe = OneHotEncoder()
    Xtrain = ohe.fit_transform(X_train[[feat_name]]).toarray()
    Xtest = ohe.transform(X_test[[feat_name]]).toarray()
    print('1')
    
    Xtrain = pd.DataFrame(Xtrain,columns = ohe.get_feature_names())
    Xtest = pd.DataFrame(Xtest,columns = ohe.get_feature_names())
    print('2')
    
    X_train = pd.concat([X_train,Xtrain],axis=1)
    X_test = pd.concat([X_test,Xtest],axis=1)
    print('3')
    
    X_train.drop(feat_name,axis=1,inplace=True)
    X_test.drop(feat_name,axis=1,inplace=True) 
    print('4')
    
    return X_train,X_test

In [149]:
X_train,X_test = oheencode('Region',X_train,X_test)

1
2
3
4


In [151]:
X_train,X_test = oheencode('seller_type',X_train,X_test)

1
2
3
4


In [152]:
X_train,X_test = oheencode('owner',X_train,X_test)

1
2
3
4


In [154]:
X_train.shape

(6324, 28)

The categorical variables 'State or Province','city' and 'company_name' has lot of levels. One hot encoding them would be a bad idea as the number of dimensions increases a lot. So lets try FeatureHash technique

In [156]:
fhState = FeatureHasher(n_features=3, input_type='string')
hash_train = fhState.fit_transform(X_train['State or Province']).toarray()
hash_test = fhState.transform(X_test['State or Province']).toarray()

Xtrain = pd.DataFrame(hash_train)
Xtest = pd.DataFrame(hash_test)

X_train = pd.concat([X_train,Xtrain],axis=1)
X_test = pd.concat([X_test,Xtest],axis=1)

X_train.drop('State or Province',axis=1,inplace=True)
X_test.drop('State or Province',axis=1,inplace=True)

In [157]:
X_train.head()

,year,selling_price,km_driven,City,transmission,mileage,engine,max_power,seats,torque_NewtonMeters,company_name,x0_CNG,x0_Diesel,x0_LPG,x0_Petrol,x0_Central,x0_East,x0_South,x0_West,x0_Dealer,x0_Individual,x0_Trustmark Dealer,x0_First Owner,x0_Fourth & Above Owner,x0_Second Owner,x0_Test Drive Car,x0_Third Owner,0,1,2
0,2018,680000,120000,Cedar Rapids,0,23.4,1248,74,5,190,Maruti,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,2015,625000,90000,Corvallis,0,23.4,1248,74,5,219,Hyundai,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,4.0,-1.0
2,2017,700000,54188,Essex,0,23.4,1248,74,7,240,Mahindra,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,4.0,3.0
3,2015,300000,71717,Lancaster,1,23.4,1248,74,5,104,Nissan,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,0.0,2.0
4,2013,1200000,110000,Olathe,1,23.4,1248,74,5,380,Audi,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,-3.0,-1.0,2.0


In [159]:
def featurehash(feat_name,X_train,X_test):
    
    fh = FeatureHasher(n_features=3, input_type='string')
    
    hash_train = fh.fit_transform(X_train[feat_name]).toarray()
    hash_test = fh.transform(X_test[feat_name]).toarray()
    print('1')
    
    Xtrain = pd.DataFrame(hash_train)
    Xtest = pd.DataFrame(hash_test)
    print('2')
    
    X_train = pd.concat([X_train,Xtrain],axis=1)
    X_test = pd.concat([X_test,Xtest],axis=1)
    print('3')
    
    X_train.drop(feat_name,axis=1,inplace=True)
    X_test.drop(feat_name,axis=1,inplace=True) 
    print('4')
    
    return X_train,X_test

In [160]:
X_train,X_test = featurehash('City',X_train,X_test)
X_train,X_test = featurehash('company_name',X_train,X_test)

1
2
3
4
1
2
3
4


In [201]:
X_train.head()

,selling_price,km_driven,transmission,mileage,engine,max_power,seats,torque_NewtonMeters
0,680000,120000,0,23.4,1248,74,5,190
1,625000,90000,0,23.4,1248,74,5,219
2,700000,54188,0,23.4,1248,74,7,240
3,300000,71717,1,23.4,1248,74,5,104
4,1200000,110000,1,23.4,1248,74,5,380


In [193]:
X_test.columns[9:]

Index(['x0_CNG', 'x0_Diesel', 'x0_LPG', 'x0_Petrol', 'x0_Central', 'x0_East',
       'x0_South', 'x0_West', 'x0_Dealer', 'x0_Individual',
       'x0_Trustmark Dealer', 'x0_First Owner', 'x0_Fourth & Above Owner',
       'x0_Second Owner', 'x0_Test Drive Car', 'x0_Third Owner'],
      dtype='object')

In [194]:
X_train.drop(X_train.columns[9:],axis = 1,inplace = True)
X_test.drop(X_test.columns[9:],axis = 1,inplace = True)

In [199]:
X_train.drop('year',axis = 1,inplace = True)
X_test.drop('year',axis = 1,inplace = True)

In [200]:
X_test.head()

,selling_price,km_driven,transmission,mileage,engine,max_power,seats,torque_NewtonMeters
0,869999,40000,0,23.4,1248,74,5,205
1,290000,15200,0,23.4,1248,74,5,72
2,370000,30000,0,23.4,1248,74,5,190
3,950000,30000,1,23.4,1248,74,5,145
4,475000,5000,1,23.4,1248,74,5,90


In [188]:
X_test.drop([0,1,2],axis = 1,inplace = True)

In [202]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

lr = LogisticRegression(solver='liblinear',penalty='l1')
lr.fit(X_train,y_train)
ypred = lr.predict(X_test)
ypredtr = lr.predict(X_train)

print('test accuracy',accuracy_score(y_test,ypred))
print('train accuracy',accuracy_score(y_train,ypredtr))
print(classification_report(y_test,ypred))

test accuracy 0.7471554993678887
train accuracy 0.7469955724225174
              precision    recall  f1-score   support

           0       0.75      1.00      0.86      1182
           1       0.00      0.00      0.00       400

    accuracy                           0.75      1582
   macro avg       0.37      0.50      0.43      1582
weighted avg       0.56      0.75      0.64      1582



C:\Users\Suraj\anaconda3\envs\tensorflowenv\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [203]:
lr = LogisticRegression()
lr.fit(X_train,y_train)
ypred = lr.predict(X_test)
ypredtr = lr.predict(X_train)

print('test accuracy',accuracy_score(y_test,ypred))
print('train accuracy',accuracy_score(y_train,ypredtr))
print(classification_report(y_test,ypred))

test accuracy 0.7471554993678887
train accuracy 0.7469955724225174
              precision    recall  f1-score   support

           0       0.75      1.00      0.86      1182
           1       0.00      0.00      0.00       400

    accuracy                           0.75      1582
   macro avg       0.37      0.50      0.43      1582
weighted avg       0.56      0.75      0.64      1582



C:\Users\Suraj\anaconda3\envs\tensorflowenv\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [204]:
set(y_test) - set(ypred)

{1}

#### Logistic Regression gave very Bad Result

In [224]:
df.head(2)

,selling_price,km_driven,Region,fuel,seller_type,transmission,owner,mileage,seats,sold,company_name
0,450000,145500,East,Diesel,Individual,0,First Owner,23.4,5,1,Maruti
1,370000,120000,East,Diesel,Individual,0,Second Owner,23.4,5,1,Skoda


In [209]:
df.drop(['year','Sales_ID','State or Province','City','torque','max_power','engine'],axis = 1,inplace = True)

In [211]:
df['mileage'] = pd.to_numeric(df['mileage'].str.split(" ")[0][0])

In [212]:
df['transmission'] = df['transmission'].map({'Manual':0,'Automatic':1})
df['sold'] = df['sold'].map({'N':0,'Y':1})

In [214]:
df['company_name'] = df['name'].apply(lambda x: x.split(" ")[0])

In [216]:
df.drop(['name'],axis = 1,inplace = True)

In [218]:
X = df.drop('sold',axis=1)
y = df['sold']

In [219]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,stratify = y,random_state=42)

In [222]:
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [225]:
def oheencode(feat_name,X_train,X_test):
    ohe = OneHotEncoder()
    Xtrain = ohe.fit_transform(X_train[[feat_name]]).toarray()
    Xtest = ohe.transform(X_test[[feat_name]]).toarray()
    print('1')
    
    Xtrain = pd.DataFrame(Xtrain,columns = ohe.get_feature_names())
    Xtest = pd.DataFrame(Xtest,columns = ohe.get_feature_names())
    print('2')
    
    X_train = pd.concat([X_train,Xtrain],axis=1)
    X_test = pd.concat([X_test,Xtest],axis=1)
    print('3')
    
    X_train.drop(feat_name,axis=1,inplace=True)
    X_test.drop(feat_name,axis=1,inplace=True) 
    print('4')
    
    return X_train,X_test

In [226]:
X_train,X_test = oheencode('Region',X_train,X_test)
X_train,X_test = oheencode('fuel',X_train,X_test)
X_train,X_test = oheencode('seller_type',X_train,X_test)
X_train,X_test = oheencode('owner',X_train,X_test)

1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4


In [231]:
X_train.head()

,selling_price,km_driven,transmission,mileage,seats,x0_Central,x0_East,x0_South,x0_West,x0_CNG,x0_Diesel,x0_LPG,x0_Petrol,x0_Dealer,x0_Individual,x0_Trustmark Dealer,x0_First Owner,x0_Fourth & Above Owner,x0_Second Owner,x0_Test Drive Car,x0_Third Owner,0,1,2,3,4,5,6,7,8,9
0,680000,120000,0,23.4,5,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,-1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,-1.0
1,625000,90000,0,23.4,5,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2,700000,54188,0,23.4,7,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,-1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,1.0
3,300000,71717,1,23.4,5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,-1.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,-2.0
4,1200000,110000,1,23.4,5,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [229]:
def featurehash(feat_name,X_train,X_test):
    
    fh = FeatureHasher(n_features=10, input_type='string')
    
    hash_train = fh.fit_transform(X_train[feat_name]).toarray()
    hash_test = fh.transform(X_test[feat_name]).toarray()
    print('1')
    
    Xtrain = pd.DataFrame(hash_train)
    Xtest = pd.DataFrame(hash_test)
    print('2')
    
    X_train = pd.concat([X_train,Xtrain],axis=1)
    X_test = pd.concat([X_test,Xtest],axis=1)
    print('3')
    
    X_train.drop(feat_name,axis=1,inplace=True)
    X_test.drop(feat_name,axis=1,inplace=True) 
    print('4')
    
    return X_train,X_test

In [230]:
X_train,X_test = featurehash('company_name',X_train,X_test)

1
2
3
4


In [232]:
lr = LogisticRegression()
lr.fit(X_train,y_train)
ypred = lr.predict(X_test)
ypredtr = lr.predict(X_train)

print('test accuracy',accuracy_score(y_test,ypred))
print('train accuracy',accuracy_score(y_train,ypredtr))
print(classification_report(y_test,ypred))

test accuracy 0.7471554993678887
train accuracy 0.7469955724225174
              precision    recall  f1-score   support

           0       0.75      1.00      0.86      1182
           1       0.00      0.00      0.00       400

    accuracy                           0.75      1582
   macro avg       0.37      0.50      0.43      1582
weighted avg       0.56      0.75      0.64      1582



C:\Users\Suraj\anaconda3\envs\tensorflowenv\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [240]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train,y_train)

ypred = neigh.predict(X_test)
ypredtr = neigh.predict(X_train)

print('test accuracy',accuracy_score(y_test,ypred))
print('train accuracy',accuracy_score(y_train,ypredtr))
print(classification_report(y_test,ypred))

test accuracy 0.7010113780025284
train accuracy 0.778146742567995
              precision    recall  f1-score   support

           0       0.75      0.90      0.82      1182
           1       0.29      0.12      0.17       400

    accuracy                           0.70      1582
   macro avg       0.52      0.51      0.50      1582
weighted avg       0.63      0.70      0.65      1582



In [234]:
from sklearn.svm import SVC

sv = SVC(class_weight = 'balanced')
sv.fit(X_train,y_train)

ypred = sv.predict(X_test)
ypredtr = sv.predict(X_train)

print('test accuracy',accuracy_score(y_test,ypred))
print('train accuracy',accuracy_score(y_train,ypredtr))
print(classification_report(y_test,ypred))

test accuracy 0.5322376738305942
train accuracy 0.5319418089816572
              precision    recall  f1-score   support

           0       0.75      0.56      0.64      1182
           1       0.26      0.46      0.33       400

    accuracy                           0.53      1582
   macro avg       0.51      0.51      0.49      1582
weighted avg       0.63      0.53      0.56      1582



In [242]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV


rf = RandomForestClassifier()

# Number of trees in random forest
n_estimators = [200,350,500]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [10,20,30]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf
               }

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   19.0s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  5.7min finished


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'max_depth': [10, 20, 30, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 350, 500]},
                   random_state=42, verbose=2)

In [243]:
rf_random.best_params_

{'n_estimators': 350,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_features': 'auto',
 'max_depth': 10}

In [ ]:
rf = RandomForestClassifier(n_estimators = 350,min_samples_split = 2,min_samples_leaf = 4,max_features = 'auto',max_depth = 10)
rf.fit(X_train,y_train)

ypred = rf.predict(X_test)
ypredtr = rf.predict(X_train)

print('test accuracy',accuracy_score(y_test,ypred))
print('train accuracy',accuracy_score(y_train,ypredtr))
print(classification_report(y_test,ypred))